In [31]:
!pip3 install matplotlib scikit-learn seaborn xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [6]:
# File paths for T1 data components
# These paths correspond to the location of CSV files containing various financial data for timestamp T1.

# Annual profit and loss data - Part 1
t1_annual_p_l_1='data/T1_data/Annual_P_L_1.csv'

# Annual profit and loss data - Part 2
t1_annual_p_l_2='data/T1_data/Annual_P_L_2.csv'

# Balance sheet data providing a snapshot of financial position at T1
t1_balance_sheet='data/T1_data/Balance_Sheet.csv'

# Quarterly profit and loss data - Part 1
t1_quarter_p_l_1='data/T1_data/Quarter_P_L_1.csv'

# Quarterly profit and loss data - Part 2
t1_quarter_p_l_2='data/T1_data/Quarter_P_L_2.csv'

# Cash flow statement detailing cash inflows and outflows at T1
t1_cash_flow_statement='data/T1_data/cash_flow_statments.csv'

# Additional metrics relevant for financial analysis at T1
t1_other_metrics='data/T1_data/other_metrics.csv'

# Stock price data for companies at T1
t1_price='data/T1_data/price.csv'

# Financial ratios - Part 1
t1_ratios_1='data/T1_data/ratios_1.csv'

# Financial ratios - Part 2
t1_ratios_2='data/T1_data/ratios_2.csv'

In [7]:
# Creating a list of DataFrames for T1 data
# This list consolidates all the loaded DataFrames corresponding to T1 financial data.

t1_df_list = [
    t1_annual_p_l_1,          # Annual Profit and Loss Data - Part 1
    t1_annual_p_l_2,          # Annual Profit and Loss Data - Part 2
    t1_balance_sheet,         # Balance Sheet Data
    t1_quarter_p_l_2,         # Quarterly Profit and Loss Data - Part 2
    t1_cash_flow_statement,   # Cash Flow Statement Data
    t1_other_metrics,         # Other Metrics Data
    t1_price,                 # Stock Price Data
    t1_ratios_1,              # Financial Ratios Data - Part 1
    t1_ratios_2               # Financial Ratios Data - Part 2
]

In [8]:
def merge_df(df_link_list):
    """
    Merges multiple DataFrames from a list of CSV file paths into a single DataFrame.
    The merging is performed iteratively using an inner join on the 'Name' column.

    Parameters:
    - df_link_list (list): List of file paths (strings) to the CSV files to be merged.

    Returns:
    - merged_df (DataFrame): The final merged DataFrame containing all unique columns from the input files.
    """

    df_list = []  # Initialize an empty list to store DataFrames.

    # Read each CSV file from the provided links and append to the df_list.
    for link in df_link_list:
        df = pd.read_csv(link)  # Load the CSV file into a DataFrame.
        df_list.append(df)      # Append the DataFrame to the list.

    merged_df = None  # Initialize the merged DataFrame.

    # Iterate through the list of DataFrames to merge them.
    for i in range(len(df_list)):
        df = df_list[i]  # Get the current DataFrame from the list.
        if merged_df is None:
            # Set the first DataFrame as the base for merging.
            merged_df = df
        else:
            # Identify columns that are not already in the merged DataFrame.
            unique_cols = [col for col in df.columns if col not in merged_df.columns or col == 'Name']

            # Perform an inner join on the 'Name' column.
            merged_df = pd.merge(merged_df, df[unique_cols], on='Name')

    return merged_df  # Return the final merged DataFrame.


In [9]:
t1_merged_df=merge_df(t1_df_list)

In [10]:
t1_merged_df_copy = t1_merged_df.copy()

In [11]:
t1_merged_df.head()

,Name,BSE Code,NSE Code,Industry,Current Price,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Historical PE 10Years,Historical PE 7Years,Historical PE 5Years,Market Capitalization 3years back,Market Capitalization 5years back,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years
0,20 Microns,533022.0,20MICRONS,Mining / Minerals / Metals,224.55,777.49,13.59,57.38,21.70,15.89,...,9.12,8.38,8.23,124.03,134.79,123.50,105.34,84.01,0.70,0.00
1,21st Cent. Mgmt.,526921.0,21STCENMGM,Finance & Investments,70.44,34.92,94.27,32.23,73.00,30.70,...,2.43,3.57,4.69,14.65,20.21,19.16,4.25,51.05,-0.35,-0.15
2,360 ONE,542772.0,360ONE,Finance & Investments,1009.40,2920.91,58.37,804.18,14.47,22.41,...,29.05,29.05,29.05,10894.33,NaN,NaN,NaN,-109.10,38.31,6.97
3,3B Blackbio,532067.0,NaN,Healthcare,1182.00,74.12,44.50,32.10,20.95,38.13,...,15.49,13.15,10.69,213.76,48.38,22.11,4.79,210.92,1.50,1.80
4,3C IT Solutions,544190.0,NaN,Computers - Software - Medium / Small,44.01,61.93,5.17,1.14,25.63,31.67,...,1.51,1.51,1.51,NaN,NaN,NaN,NaN,64.30,NaN,NaN


In [12]:
missing_values = t1_merged_df.isna().sum()
missing_percentage = (missing_values / len(t1_merged_df)) * 100

# Show features with missing data
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_data = missing_data[missing_data['Percentage'] <= 10]
print(missing_data)

                                     Missing Values  Percentage
Name                                              0    0.000000
BSE Code                                        464    9.937888
Industry                                          0    0.000000
Current Price                                     1    0.021418
Sales                                             4    0.085671
...                                             ...         ...
Return on assets 3years                         219    4.690512
Historical PE 10Years                           182    3.898051
Historical PE 7Years                            255    5.461555
Historical PE 5Years                            339    7.260655
Average Working Capital Days 3years               0    0.000000

[222 rows x 2 columns]


In [13]:
# Links to the CSV files containing the financial data for T2.

# Annual Profit & Loss statements for T2 (split into two parts).
t2_annual_p_l_1 = 'data/T2_data/Annual_P_L_1.csv'
t2_annual_p_l_2 = 'data/T2_data/Annual_P_L_2.csv'

# Balance sheet data for T2.
t2_balance_sheet = 'data/T2_data/Balance_Sheet.csv'

# Quarterly Profit & Loss statements for T2 (split into two parts).
t2_quarter_p_l_1 = 'data/T2_data/Quarter_P_L_1.csv'
t2_quarter_p_l_2 = 'data/T2_data/Quarter_P_L_2.csv'

# Cash flow statement for T2.
t2_cash_flow_statement = 'data/T2_data/cash_flow_statements.csv'

# Other metrics for T2.
t2_other_metrics = 'data/T2_data/other_metrics.csv'

# Price data for T2 (stocks or company valuation).
t2_price = 'data/T2_data/price.csv'

# Financial ratios data for T2 (split into two parts).
t2_ratios_1 = 'data/T2_data/ratios_1.csv'
t2_ratios_2 = 'data/T2_data/ratios_2.csv'

In [14]:
# List of links to the CSV files containing the financial data for T2.

t2_df_list = [
    t2_annual_p_l_1,          # Annual Profit & Loss Statement Part 1.
    t2_annual_p_l_2,          # Annual Profit & Loss Statement Part 2.
    t2_balance_sheet,         # Balance Sheet Data.
    t2_quarter_p_l_2,         # Quarterly Profit & Loss Statement Part 2.
    t2_cash_flow_statement,   # Cash Flow Statement Data.
    t2_other_metrics,         # Additional Financial Metrics.
    t2_price,                 # Price Data (Stock/Valuation).
    t2_ratios_1,              # Financial Ratios Part 1.
    t2_ratios_2               # Financial Ratios Part 2.
]

In [15]:
t2_merged_df=merge_df(t2_df_list)

In [16]:
#Selecting common coloumns between t1 and t2
common_columns = t1_merged_df.columns.intersection(t2_merged_df.columns)


In [17]:
t1_merged_df=t1_merged_df[common_columns]
t2_merged_df=t2_merged_df[common_columns]

In [18]:
#Removing coloumns where current price is not present
t1_merged_df = t1_merged_df[t1_merged_df['Current Price'].notna()]
t2_merged_df = t2_merged_df[t2_merged_df['Current Price'].notna()]

In [19]:
t1_merged_df.shape

(4668, 316)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
#Use this to drop the given coloumns
drop_columns = [
    'Current Price',                # Target variable to be predicted
    "Name",                         # Likely an identifier, not useful for prediction
    "BSE Code",                     # Not needed for prediction
    "NSE Code",                     # Not needed for prediction
    "Credit rating"
]

In [22]:
#Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(t1_merged_df.drop(drop_columns, axis=1), t1_merged_df['Current Price'], test_size=0.2, random_state=42)

In [23]:
#Splitting t2 data in train and test
X_t2,y_t2=t2_merged_df.drop(drop_columns, axis=1),t2_merged_df['Current Price']

In [24]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression, SelectPercentile
from sklearn.metrics import r2_score

In [25]:
# Step 1: Specify the columns for processing
ohe_columns =['Industry']
# These columns will be One-Hot Encoded as they contain categorical data.
#other_columns = X_train.columns.difference(ohe_columns) #This was incorrectly taking all coloumns
other_columns = X_train.columns.difference(ohe_columns).tolist() #This will take all the numerical coloumns only
# Step 2: Define the preprocessing pipeline
preprocessor = ColumnTransformer([
    ('si_imputer', SimpleImputer(strategy='median'), other_columns),  # Apply Simple Imputer for numeric columns
    ('ohe', OneHotEncoder(handle_unknown='ignore',sparse_output=False), ohe_columns), # Apply One-Hot Encoding for categorical columns

], remainder='passthrough')

In [26]:
#Selecting important coloumns for training
sp = SelectPercentile(score_func=mutual_info_regression, percentile=(75))

In [33]:
from xgboost import XGBRegressor

In [34]:
best_params_xgb = {
    'random_state': 42,  # Seed for reproducibility
    'n_estimators': 50,  # Number of estimators
    'max_leaves': 2,     # Maximum number of leaves per tree
    'max_depth': 3,      # Maximum depth of a tree
    'lambda': 10,        # Regularization term (L2)
    'eta': 0.6,          # Learning rate for XGBoost
    'alpha': 0           # Regularization term (L1)
}

In [35]:
best_params_xgb = {
    'random_state':42,
    'n_estimators':300,
    'max_leaves':10,
    'max_depth':4,
    'reg_lambda':5,
    'learning_rate':0.2,
    'reg_alpha':0.1
}

In [36]:
xgb_pipe = Pipeline([
    ('preprocessor', preprocessor),  # Preprocessing for imputation and encoding
    ('sp', sp),                      # Feature selection using Mutual Information
    ('model', XGBRegressor(**best_params_xgb))  # XGBoost model with custom parameters
])

In [37]:
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_imputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['52w Index',
                                                   'Accumulated depreciation',
                                                   'Advance from Customers',
                                                   'Altman Z Score',
                                                   'Asset Turnover Ratio',
                                                   'Average 5years dividend',
                                                   'Average EBIT 10Year',
                                                   'Average EBIT 5Year',
                                                   'Average Earnings 10Year',
                                                   'Average Earnings 5Year',
                                                   'Average...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=10, min_child_weight=None,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=300,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=42, ...))])

In [38]:
# Predict the target variable for the training dataset using the trained pipeline
y_pred_xgb_train = xgb_pipe.predict(X_train)

# Calculate the R² score for the training predictions
r2_xgb_train = r2_score(y_pred_xgb_train, y_train)

# Display the R² score for the training dataset
print(f"R² Score (XGBoost - Training Data): {r2_xgb_train:.4f}")

R² Score (XGBoost - Training Data): 1.0000


In [39]:
y_pred_xgb_test = xgb_pipe.predict(X_test)
r2_xgb_test=r2_score(y_pred_xgb_test,y_test.fillna(0))
print(f"R² Score (XGBoost - Test Data): {r2_xgb_test:.4f}")

R² Score (XGBoost - Test Data): 0.7730


In [40]:
y_pred_xgb_t2 = xgb_pipe.predict(X_t2)
r2_xgb_t2=r2_score(y_pred_xgb_t2,y_t2.fillna(0))
print(f"R² Score (XGBoost - T2 Data): {r2_xgb_t2:.4f}")

R² Score (XGBoost - T2 Data): 0.7102


In [41]:
from sklearn.ensemble import GradientBoostingRegressor

In [42]:
# Learning rate for boosting
gb_pipe = Pipeline([
    ('preprocessor', preprocessor),  # Impute missing values and encode categorical features
    ('sp', sp),                      # Select top features based on Mutual Information scores
    ('model', GradientBoostingRegressor(
        loss='squared_error',        # Use squared error as the loss function
        learning_rate=0.1,           # Learning rate for boosting
        n_estimators=300,            # Number of boosting iterations
        random_state=42              # Ensure reproducibility
    ))
])

In [43]:
#Traning the model
gb_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_imputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['52w Index',
                                                   'Accumulated depreciation',
                                                   'Advance from Customers',
                                                   'Altman Z Score',
                                                   'Asset Turnover Ratio',
                                                   'Average 5years dividend',
                                                   'Average EBIT 10Year',
                                                   'Average EBIT 5Year',
                                                   'Average Earnings 10Year',
                                                   'Average Earnings 5Year',
                                                   'Average...
                                                   'Book value 5years back',
                                                   'Book value of unquoted '
                                                   'investments',
                                                   'Book value preceding year',
                                                   'CROIC',
                                                   'Capital work in progress', ...]),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Industry'])])),
                ('sp',
                 SelectPercentile(percentile=75,
                                  score_func=<function mutual_info_regression at 0x306ee9d30>)),
                ('model',
                 GradientBoostingRegressor(n_estimators=300, random_state=42))])

In [44]:
# Predict the target variable for the training dataset using the trained pipeline
y_pred_gb_train = gb_pipe.predict(X_train)

# Calculate the R² score for the training predictions
r2_gb_train = r2_score(y_pred_gb_train, y_train)

# Display the R² score for the training dataset
print(f"R² Score (XGBoost - Training Data): {r2_gb_train:.4f}")

R² Score (XGBoost - Training Data): 1.0000


In [45]:
#Testing on Validation set
y_pred_gb_test = gb_pipe.predict(X_test)
r2_gb_test=r2_score(y_pred_gb_test,y_test.fillna(0))
print(f"R² Score (XGBoost - Test Data): {r2_gb_test:.4f}")

R² Score (XGBoost - Test Data): 0.9965


In [46]:
#Testing on T2 data
y_pred_gb_t2 = gb_pipe.predict(X_t2)
r2_gb_t2=r2_score(y_pred_gb_t2,y_t2.fillna(0))
print(f"R² Score (XGBoost - T2 Data): {r2_gb_t2:.4f}")

R² Score (XGBoost - T2 Data): 0.9907


In [47]:
#Saving the predicted price in t2 dataset
t2_merged_df['Predicted Price'] = y_pred_gb_t2

In [48]:
#Mergeing t1 current price in t2 dataset
t2_merged_df = pd.merge(t2_merged_df, t1_merged_df_copy[['Name','Current Price']], on='Name')

In [49]:
t2_merged_df

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Historical PE 5Years,Market Capitalization 3years back,Market Capitalization 5years back,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y
0,20 Microns,533022.0,20MICRONS,Mining / Minerals / Metals,287.75,817.48,13.30,60.12,21.70,16.72,...,8.39,124.03,134.79,123.50,105.34,84.01,0.75,0.06,294.185709,224.55
1,21st Cent. Mgmt.,526921.0,21STCENMGM,Finance & Investments,86.30,146.13,42.95,59.40,78.43,56.57,...,2.09,14.65,20.21,19.16,4.25,172.84,-0.35,-0.15,81.614163,70.44
2,360 ONE,542772.0,360ONE,Finance & Investments,1056.30,2831.50,55.18,926.34,14.47,24.04,...,29.31,10894.33,NaN,NaN,NaN,-109.10,39.65,7.35,1119.257692,1009.40
3,3B Blackbio,532067.0,NaN,Healthcare,1245.65,74.12,44.50,32.10,20.95,38.13,...,10.86,213.76,48.38,22.11,4.79,210.92,1.81,1.87,1142.187437,1182.00
4,3C IT Solutions,544190.0,NaN,Computers - Software - Medium / Small,39.30,29.84,7.44,0.12,9.60,0.28,...,132.14,NaN,NaN,NaN,NaN,122.43,NaN,NaN,41.793570,44.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4608,Zota Health Care,NaN,ZOTA,Trading,690.75,180.49,4.18,-14.35,-7.32,-5.55,...,133.91,340.16,456.33,NaN,NaN,129.70,0.01,0.00,727.568891,571.75
4609,Zuari Agro Chem.,534742.0,ZUARI,Fertilizers,221.91,4486.73,7.82,133.24,11.13,31.68,...,4.38,382.31,781.23,1529.44,538.34,26.92,0.18,-2.02,197.161641,227.50
4610,Zuari Industries,500780.0,ZUARIIND,Sugar,373.95,837.80,6.19,-0.71,4.98,239.72,...,25.55,236.67,318.11,374.26,221.97,262.15,0.36,-3.42,395.706218,398.00
4611,Zydus Lifesci.,532321.0,ZYDUSLIFE,Pharmaceuticals - Indian - Bulk Drugs & Formln,1249.90,19547.40,27.54,3860.03,22.34,38.36,...,22.98,45136.81,35528.99,45249.42,20984.83,103.20,0.94,2.06,1278.549320,1070.05


In [50]:
#Calculating predicted rate of return
t2_merged_df['Predicted rate of return'] = (t2_merged_df['Predicted Price'] - t2_merged_df['Current Price_y']) / t2_merged_df['Current Price_y']

In [51]:
#Converting it into percentage
t2_merged_df['Predicted rate of return'] = t2_merged_df['Predicted rate of return'] * 100

In [52]:
#Creating another dataframe with return rate > 0
project_df = t2_merged_df[t2_merged_df['Predicted rate of return']>0]

In [53]:
def classify_risk(row):
    if row["Debt to equity"] < 0.5 and 10 <= row["Price to Earning"] <= 25 and row["Return on equity"] > 15:
        return "Low Risk"
    elif 0.5 <= row["Debt to equity"] <= 1.5 and 25 <= row["Price to Earning"] <= 50 and 10 <= row["Return on equity"] <= 15:
        return "Medium Risk"
    else:
        return "High Risk"

In [54]:
project_df['Risk Level'] = project_df.apply(classify_risk, axis=1)

/var/folders/s5/q_8xj8pj4sldcg_1vccll_n00000gn/T/ipykernel_1647/2728636457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  project_df['Risk Level'] = project_df.apply(classify_risk, axis=1)


In [55]:
#Sorting in descending value
project_df = project_df.sort_values(by='Predicted rate of return', ascending=False)
project_df

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 5years back,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level
1994,K-Lifestyle,514221.0,NaN,Textiles - Products,0.37,2.70,15.19,-56.75,-5.85,-0.56,...,20.45,27.60,106.33,-8586.18,NaN,NaN,5.652429,0.33,1612.857131,High Risk
4008,Sword-Edge Comm.,512359.0,NaN,Trading,0.34,1.04,57.69,0.59,-1.03,0.03,...,1.52,7.81,17.14,13568.48,0.00,0.00,4.090482,0.33,1139.539936,High Risk
4477,VKJ Infradevelop,536128.0,NaN,Construction,0.52,0.00,NaN,0.02,0.07,0.00,...,161.13,98.29,NaN,0.00,0.00,0.00,5.253046,0.46,1041.966597,High Risk
3470,Sanwaria Consum.,519260.0,SANWARIA,Food - Processing - Indian,0.39,1.07,-71.03,-5.80,-2.08,-0.08,...,669.85,294.44,238.76,77826.87,0.00,0.00,4.388992,0.41,970.485963,High Risk
4212,Triton Corp.,523387.0,NaN,Computers - Software - Medium / Small,0.54,0.00,NaN,-0.44,-12.85,-0.02,...,3.80,7.00,5.00,0.00,0.00,-0.10,5.384179,0.52,935.419058,High Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Modella Woollens,503772.0,NaN,Trading,69.70,0.00,NaN,-0.13,NaN,-1.43,...,0.99,0.52,1.90,0.00,0.00,-0.97,71.021516,71.00,0.030304,High Risk
2624,Naksh Precious,539402.0,NaN,Trading,9.31,0.49,-4.08,0.03,-0.66,0.03,...,13.71,14.93,NaN,10170.40,0.00,-1.40,8.712036,8.71,0.023379,High Risk
2787,Olympic Oil Ind.,507609.0,NaN,Trading,49.98,0.00,NaN,-0.46,-0.91,-1.61,...,12.27,21.38,0.62,0.00,0.00,0.00,50.351625,50.34,0.023093,High Risk
2940,Pentagon Rubber,NaN,PENTAGON,Miscellaneous,118.25,57.77,12.01,4.44,27.41,5.76,...,NaN,NaN,NaN,174.28,NaN,NaN,117.015682,117.00,0.013403,High Risk


In [56]:
project_df[project_df['Risk Level']=='Low Risk']

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 5years back,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level
346,Ashika Credit,543766.0,NaN,Finance & Investments,177.75,32.02,82.26,20.92,17.92,17.61,...,32.66,20.08,45.64,-9.62,-2.75,0.00,184.952479,83.79,120.733356,Low Risk
1083,Edvenswa Enter,517170.0,NaN,Computers - Software - Medium / Small,95.89,84.29,13.68,7.38,23.81,3.90,...,5.04,3.62,3.22,125.40,0.36,0.00,102.542637,58.20,76.190098,Low Risk
3843,SRM Contractors,544158.0,SRM,Construction,269.50,342.42,12.16,26.97,28.05,16.11,...,NaN,NaN,NaN,55.46,NaN,NaN,305.274404,179.55,70.021946,Low Risk
3524,Seacoast Ship.,542753.0,NaN,Shipping,6.44,374.16,9.39,20.29,28.33,0.38,...,NaN,NaN,NaN,143.53,-1.41,0.00,8.401587,4.95,69.729030,Low Risk
4303,Upsurge Invest.,531390.0,NaN,Finance & Investments,99.18,58.06,22.99,11.27,29.48,7.44,...,29.93,22.35,23.56,94.55,0.00,0.00,101.713430,61.76,64.691434,Low Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2783,Olatech Solution,543578.0,NaN,Computers - Software - Medium / Small,152.95,15.16,28.76,3.02,91.06,6.98,...,NaN,NaN,NaN,88.15,NaN,NaN,151.134165,149.45,1.126909,Low Risk
509,Baweja Studios L,NaN,BAWEJA,Entertainment / Electronic Media Software,89.30,64.30,17.57,8.22,53.78,4.46,...,NaN,NaN,NaN,72.96,NaN,NaN,85.287131,84.75,0.633783,Low Risk
1412,Godfrey Phillips,500163.0,GODFRYPHLP,Cigarettes,4183.15,4419.59,20.19,883.04,22.62,169.84,...,6026.61,5891.43,3333.46,50.07,0.57,0.23,4287.074920,4262.00,0.588337,Low Risk
3959,Suraj Products,518075.0,NaN,Steel - Medium / Small,622.55,344.68,14.76,28.93,25.99,25.38,...,22.63,15.16,14.82,84.09,0.00,0.02,663.331102,661.10,0.337483,Low Risk


In [57]:
project_df_1 = project_df.copy()

In [58]:
#Calculating real rate of return
project_df['Real rate of return'] = (t2_merged_df['Current Price_x'] - t2_merged_df['Current Price_y']) / t2_merged_df['Current Price_y']

In [59]:
project_df['Real rate of return'] = project_df['Real rate of return'] * 100

In [60]:
project_df = project_df[project_df['Real rate of return']>0]

In [61]:
#Filtering out stocks with rate of return > 150
project_df_11 = project_df[project_df['Predicted rate of return']<=150]

In [62]:
project_df[project_df['Predicted rate of return']>=150]

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level,Real rate of return
1994,K-Lifestyle,514221.0,NaN,Textiles - Products,0.37,2.70,15.19,-56.75,-5.85,-0.56,...,27.60,106.33,-8586.18,NaN,NaN,5.652429,0.33,1612.857131,High Risk,12.121212
4008,Sword-Edge Comm.,512359.0,NaN,Trading,0.34,1.04,57.69,0.59,-1.03,0.03,...,7.81,17.14,13568.48,0.00,0.00,4.090482,0.33,1139.539936,High Risk,3.030303
4477,VKJ Infradevelop,536128.0,NaN,Construction,0.52,0.00,NaN,0.02,0.07,0.00,...,98.29,NaN,0.00,0.00,0.00,5.253046,0.46,1041.966597,High Risk,13.043478
4212,Triton Corp.,523387.0,NaN,Computers - Software - Medium / Small,0.54,0.00,NaN,-0.44,-12.85,-0.02,...,7.00,5.00,0.00,0.00,-0.10,5.384179,0.52,935.419058,High Risk,3.846154
3656,Shree Securities,538975.0,NaN,Finance & Investments,0.37,0.71,-1908.45,-13.55,-82.93,-0.74,...,NaN,NaN,-235.61,0.00,0.00,3.423530,0.36,850.980545,High Risk,2.777778
4453,Visesh Infotec.,532411.0,VISESHINFO,Computers - Software - Medium / Small,0.42,0.54,-292.59,-3.56,-0.88,-0.01,...,75.49,94.36,50693.79,0.00,0.00,3.795901,0.40,848.975249,High Risk,5.000000
3735,Siti Networks,532795.0,SITINET,Entertainment / Electronic Media Software,0.88,1294.31,4.03,-292.18,-49.90,-3.42,...,2815.25,1051.39,-164.99,-10.26,2.43,5.445210,0.65,737.724556,High Risk,35.384615
784,Cistro Telelink,531775.0,NaN,Trading,0.67,0.24,-29.17,0.02,0.54,-0.28,...,1.03,0.36,-447.12,0.00,0.00,4.653249,0.64,627.070140,High Risk,4.687500
1188,Excel Realty,533090.0,EXCEL,Construction,0.84,2.42,-95.04,1.36,0.68,0.01,...,114.21,63.45,5732.26,0.00,0.00,4.704953,0.71,562.669429,High Risk,18.309859
3593,Sharanam Infra,539584.0,NaN,Trading,0.85,1.76,2.27,0.03,0.40,0.00,...,22.80,NaN,674.70,0.00,0.00,3.319058,0.55,503.465109,High Risk,54.545455


In [63]:
project_df_11.head(10)

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level,Real rate of return
3622,Shine Fashions,543244.0,NaN,Trading,440.00,54.13,9.75,3.91,53.98,13.97,...,NaN,NaN,126.80,0.00,0.00,438.631670,177.00,147.814503,High Risk,148.587571
985,Dhenu Buildcon,501945.0,NaN,Finance & Investments,2.84,0.00,NaN,-0.10,-4.32,-0.05,...,5.12,5.22,0.00,0.00,0.00,5.966878,2.41,147.588317,High Risk,17.842324
4583,Yuvraaj Hygiene,531663.0,NaN,Plastics Products,1.86,18.97,-1.27,-0.34,3.27,-0.04,...,5.00,17.88,22.63,0.00,0.00,4.324094,1.78,142.926651,High Risk,4.494382
3793,Source Industrie,521036.0,NaN,Textiles - Processing,3.17,0.07,-214.29,-0.15,-3.28,-0.13,...,1.62,22.92,18263.90,0.00,0.00,6.125087,2.53,142.098300,High Risk,25.296443
1384,Glittek Granites,513528.0,NaN,Miscellaneous,4.88,6.93,-59.16,-6.14,-15.19,-2.37,...,4.80,13.76,955.16,0.00,-0.39,7.849593,3.25,141.525942,High Risk,50.153846
1119,Emerald Finance,538882.0,NaN,Finance & Investments,71.40,14.94,57.16,4.98,12.39,1.65,...,58.64,NaN,604.35,1.31,0.00,75.206282,31.20,141.045777,High Risk,128.846154
653,Brijlaxmi Leas.,532113.0,NaN,Finance & Investments,15.95,3.18,72.33,0.76,10.83,1.18,...,1.12,2.35,-46.70,0.00,0.00,18.795835,7.80,140.972241,High Risk,104.487179
2095,Khyati Multimed.,531692.0,NaN,Entertainment / Electronic Media Software,4.26,0.96,12.50,0.09,5.13,0.08,...,0.86,0.89,54.75,0.00,0.00,6.253419,2.66,135.090944,High Risk,60.150376
3178,Rajasthan Gases,526873.0,NaN,Miscellaneous,42.99,0.00,NaN,-0.50,-0.58,-0.07,...,2.63,2.59,0.00,0.00,0.00,52.217470,22.32,133.949239,High Risk,92.607527
1754,Indo Credit Cap.,526887.0,NaN,Finance & Investments,2.14,0.06,-266.67,-0.03,0.17,-0.04,...,0.80,0.70,-7876.29,0.00,0.00,4.552404,1.95,133.456624,High Risk,9.743590


In [64]:
#Filtering out stocks with current price less than 10
project_df_11 = project_df_11[project_df_11['Current Price_y']>10]

In [65]:
project_df_11.head(10)

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 7years back,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level,Real rate of return
3622,Shine Fashions,543244.0,NaN,Trading,440.00,54.13,9.75,3.91,53.98,13.97,...,NaN,NaN,126.80,0.00,0.0,438.631670,177.00,147.814503,High Risk,148.587571
1119,Emerald Finance,538882.0,NaN,Finance & Investments,71.40,14.94,57.16,4.98,12.39,1.65,...,58.64,NaN,604.35,1.31,0.0,75.206282,31.20,141.045777,High Risk,128.846154
3178,Rajasthan Gases,526873.0,NaN,Miscellaneous,42.99,0.00,NaN,-0.50,-0.58,-0.07,...,2.63,2.59,0.00,0.00,0.0,52.217470,22.32,133.949239,High Risk,92.607527
3897,Sudarshan Pharma,543828.0,NaN,Trading,179.60,465.56,4.86,11.36,17.86,4.72,...,NaN,NaN,64.88,NaN,NaN,187.984343,81.30,131.223054,Medium Risk,120.910209
843,Continental Seed,NaN,CONTI,Trading,51.00,97.68,-0.07,-2.25,-9.96,0.67,...,NaN,NaN,-4.37,0.00,0.0,55.325054,24.00,130.521057,High Risk,112.500000
2700,New Light Appare,540243.0,NaN,Trading,39.24,12.05,2.66,0.37,6.05,1.68,...,NaN,NaN,304.06,4.19,0.0,44.687453,20.00,123.437263,High Risk,96.200000
1809,Integ. Proteins,519606.0,NaN,Solvent Extraction,32.35,0.24,-50.00,0.09,3.73,0.24,...,0.82,0.72,-73.36,0.00,0.0,35.364366,15.83,123.400920,High Risk,104.358812
2478,Mercury Trade Li,512415.0,NaN,Trading,26.39,13.83,10.27,1.14,23.61,4.19,...,9.58,9.58,122.27,22.04,0.0,36.648382,16.50,122.111408,High Risk,59.939394
346,Ashika Credit,543766.0,NaN,Finance & Investments,177.75,32.02,82.26,20.92,17.92,17.61,...,20.08,45.64,-9.62,-2.75,0.0,184.952479,83.79,120.733356,Low Risk,112.137487
3873,Stellant Secu.,526071.0,NaN,Finance & Investments,31.97,0.41,68.29,0.29,25.17,3.92,...,0.27,4.14,1.52,0.00,0.0,37.493493,17.49,114.371030,High Risk,82.790166


In [66]:
import scipy.optimize as opt

In [67]:
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

In [68]:
project_df_12 = project_df_11.copy()

In [69]:
project_df_12['predicted rate'] = project_df_12['Predicted rate of return']/100


In [70]:
project_df_12

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Market Capitalization 10years back,Average Working Capital Days 3years,Change in FII holding 3Years,Change in DII holding 3Years,Predicted Price,Current Price_y,Predicted rate of return,Risk Level,Real rate of return,predicted rate
3622,Shine Fashions,543244.0,NaN,Trading,440.00,54.13,9.75,3.91,53.98,13.97,...,NaN,126.80,0.00,0.00,438.631670,177.00,147.814503,High Risk,148.587571,1.478145
1119,Emerald Finance,538882.0,NaN,Finance & Investments,71.40,14.94,57.16,4.98,12.39,1.65,...,NaN,604.35,1.31,0.00,75.206282,31.20,141.045777,High Risk,128.846154,1.410458
3178,Rajasthan Gases,526873.0,NaN,Miscellaneous,42.99,0.00,NaN,-0.50,-0.58,-0.07,...,2.59,0.00,0.00,0.00,52.217470,22.32,133.949239,High Risk,92.607527,1.339492
3897,Sudarshan Pharma,543828.0,NaN,Trading,179.60,465.56,4.86,11.36,17.86,4.72,...,NaN,64.88,NaN,NaN,187.984343,81.30,131.223054,Medium Risk,120.910209,1.312231
843,Continental Seed,NaN,CONTI,Trading,51.00,97.68,-0.07,-2.25,-9.96,0.67,...,NaN,-4.37,0.00,0.00,55.325054,24.00,130.521057,High Risk,112.500000,1.305211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,Medi Assist Ser.,544088.0,MEDIASSIST,Miscellaneous,582.65,634.73,21.00,84.37,23.54,9.53,...,NaN,-34.60,NaN,NaN,552.596698,552.30,0.053720,High Risk,5.495202,0.000537
1595,Hil Ltd,509675.0,HIL,Cement Products,2944.50,3374.97,3.72,32.96,1.89,46.14,...,221.83,39.75,-1.59,-2.00,2921.364727,2919.95,0.048450,High Risk,0.840768,0.000485
1406,Go Fashion (I),543401.0,GOCOLORS,Textiles - Products,1127.35,792.81,31.55,85.14,15.66,15.76,...,NaN,161.06,NaN,NaN,1031.703222,1031.30,0.039098,High Risk,9.313488,0.000391
2940,Pentagon Rubber,NaN,PENTAGON,Miscellaneous,118.25,57.77,12.01,4.44,27.41,5.76,...,NaN,174.28,NaN,NaN,117.015682,117.00,0.013403,High Risk,1.068376,0.000134


In [71]:
#Normilising differnt important ratios for stock selection
project_df_12["Norm Return"] = normalize(project_df_12["predicted rate"])
project_df_12['norm roe'] = normalize(project_df_12['Return on equity'])
project_df_12['norm pe'] = normalize(project_df_12['Price to Earning'])
project_df_12['norm roa'] = normalize(project_df_12['Return on assets'])



In [72]:
#Calculating final score for each stock
project_df_12["Final Score"] = (0.4 * project_df_12["Norm Return"] +
                     0.3 * project_df_12["norm roe"] +
                     0.2 * project_df_12["norm roa"] +
                     0.1 * project_df_12["norm pe"])

In [73]:
#Selecting 50 stocks with largest final score
top_50_stocks = project_df_12.nlargest(50, "Final Score")

In [74]:
top_50_stocks

,Name,BSE Code,NSE Code,Industry,Current Price_x,Sales,OPM,Profit after tax,Return on capital employed,EPS,...,Current Price_y,Predicted rate of return,Risk Level,Real rate of return,predicted rate,Norm Return,norm roe,norm pe,norm roa,Final Score
1545,Harshdeep,544105.0,NaN,Miscellaneous,71.32,48.25,21.99,6.55,18600.00,4.07,...,62.00,16.528918,Low Risk,15.032258,0.165289,0.111753,1.000000,0.001131,1.000000,0.544814
3622,Shine Fashions,543244.0,NaN,Trading,440.00,54.13,9.75,3.91,53.98,13.97,...,177.00,147.814503,High Risk,148.587571,1.478145,1.000000,0.104780,0.002076,0.025083,0.436658
1119,Emerald Finance,538882.0,NaN,Finance & Investments,71.40,14.94,57.16,4.98,12.39,1.65,...,31.20,141.045777,High Risk,128.846154,1.410458,0.954204,0.102152,0.002868,0.024063,0.417427
3897,Sudarshan Pharma,543828.0,NaN,Trading,179.60,465.56,4.86,11.36,17.86,4.72,...,81.30,131.223054,Medium Risk,120.910209,1.312231,0.887746,0.102295,0.002519,0.023903,0.390819
1809,Integ. Proteins,519606.0,NaN,Solvent Extraction,32.35,0.24,-50.00,0.09,3.73,0.24,...,15.83,123.400920,High Risk,104.358812,1.234009,0.834823,0.101764,0.008876,0.023780,0.370102
2700,New Light Appare,540243.0,NaN,Trading,39.24,12.05,2.66,0.37,6.05,1.68,...,20.00,123.437263,High Risk,96.200000,1.234373,0.835069,0.102532,0.005554,0.023782,0.370099
2478,Mercury Trade Li,512415.0,NaN,Trading,26.39,13.83,10.27,1.14,23.61,4.19,...,16.50,122.111408,High Risk,59.939394,1.221114,0.826099,0.102841,0.000373,0.024374,0.366204
346,Ashika Credit,543766.0,NaN,Finance & Investments,177.75,32.02,82.26,20.92,17.92,17.61,...,83.79,120.733356,Low Risk,112.137487,1.207334,0.816775,0.102693,0.000629,0.024575,0.362496
3873,Stellant Secu.,526071.0,NaN,Finance & Investments,31.97,0.41,68.29,0.29,25.17,3.92,...,17.49,114.371030,High Risk,82.790166,1.143710,0.773729,0.104815,0.000499,0.025322,0.346051
4369,Vaswani Industri,533576.0,VASWANI,Steel - Sponge Iron,56.23,389.34,5.69,9.03,13.23,3.01,...,34.90,109.755533,High Risk,61.117479,1.097555,0.742502,0.102052,0.001210,0.023884,0.332514


In [75]:
top_50_stocks = top_50_stocks[["Name", "Current Price_x", "Current Price_y", "Predicted Price", 'BSE Code', 'NSE Code']]

In [76]:
top_50_stocks

,Name,Current Price_x,Current Price_y,Predicted Price,BSE Code,NSE Code
1545,Harshdeep,71.32,62.00,72.247929,544105.0,NaN
3622,Shine Fashions,440.00,177.00,438.631670,543244.0,NaN
1119,Emerald Finance,71.40,31.20,75.206282,538882.0,NaN
3897,Sudarshan Pharma,179.60,81.30,187.984343,543828.0,NaN
1809,Integ. Proteins,32.35,15.83,35.364366,519606.0,NaN
2700,New Light Appare,39.24,20.00,44.687453,540243.0,NaN
2478,Mercury Trade Li,26.39,16.50,36.648382,512415.0,NaN
346,Ashika Credit,177.75,83.79,184.952479,543766.0,NaN
3873,Stellant Secu.,31.97,17.49,37.493493,526071.0,NaN
4369,Vaswani Industri,56.23,34.90,73.204681,533576.0,VASWANI


In [77]:
top_50_stocks['Invest'] = 1000000/len(top_50_stocks)
top_50_stocks['Unit'] = (top_50_stocks['Invest']/top_50_stocks['Current Price_y']).astype(int)

/var/folders/s5/q_8xj8pj4sldcg_1vccll_n00000gn/T/ipykernel_1647/2566844541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_stocks['Invest'] = 1000000/len(top_50_stocks)
/var/folders/s5/q_8xj8pj4sldcg_1vccll_n00000gn/T/ipykernel_1647/2566844541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_stocks['Unit'] = (top_50_stocks['Invest']/top_50_stocks['Current Price_y']).astype(int)


In [78]:
(top_50_stocks['Unit']*top_50_stocks['Current Price_x']).sum()

1691615.95

In [80]:
top_50_stocks.to_csv('top_50_stocks.csv', index=False)